In [39]:
# !pip install -r requirements.txt
# !python -m spacy download en_core_web_sm

In [40]:
import pandas as pd 
import string
import re
import numpy as np
import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords 
from nltk.tokenize import TweetTokenizer
import emot
emot_obj = emot.emot()
import spacy
from tools_gdg import *


In [41]:
data = pd.read_csv("../data/amazon.csv")

In [42]:
data.drop(columns = ['discount_percentage','img_link','product_name','product_link','user_name','review_id','about_product'], inplace = True)

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        1465 non-null   object
 1   category          1465 non-null   object
 2   discounted_price  1465 non-null   object
 3   actual_price      1465 non-null   object
 4   rating            1465 non-null   object
 5   rating_count      1463 non-null   object
 6   user_id           1465 non-null   object
 7   review_title      1465 non-null   object
 8   review_content    1465 non-null   object
dtypes: object(9)
memory usage: 103.1+ KB


In [44]:
data.head()

,product_id,category,discounted_price,actual_price,rating,rating_count,user_id,review_title,review_content
0,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",4.2,"24,269","AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...
1,B098NS6PVG,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,4.0,"43,994","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...
2,B096MSW6CT,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",3.9,"7,928","AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a..."
3,B08HDJ86NZ,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,4.2,"94,363","AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou..."
4,B08CF3B7N1,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,4.2,"16,905","AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th..."


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        1465 non-null   object
 1   category          1465 non-null   object
 2   discounted_price  1465 non-null   object
 3   actual_price      1465 non-null   object
 4   rating            1465 non-null   object
 5   rating_count      1463 non-null   object
 6   user_id           1465 non-null   object
 7   review_title      1465 non-null   object
 8   review_content    1465 non-null   object
dtypes: object(9)
memory usage: 103.1+ KB


In [46]:
def convert(d: str):
  x = str(data[d][i])
  st = ""
  for j in x:
    if (j >= '0' and j <= '9') or j == '.':
      st += j
  return st

lis = ["discounted_price", "actual_price", "rating", "rating_count"]

for i in range(len(data)):
  for d in lis:
    x = convert(d)
    data.loc[i, d] = x
    if len(x) > 0:
      data.loc[i, d] = float(x)

In [47]:
data.head()

,product_id,category,discounted_price,actual_price,rating,rating_count,user_id,review_title,review_content
0,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...
1,B098NS6PVG,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,4.0,43994.0,"AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...
2,B096MSW6CT,Computers&Accessories|Accessories&Peripherals|...,199.0,1899.0,3.9,7928.0,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a..."
3,B08HDJ86NZ,Computers&Accessories|Accessories&Peripherals|...,329.0,699.0,4.2,94363.0,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou..."
4,B08CF3B7N1,Computers&Accessories|Accessories&Peripherals|...,154.0,399.0,4.2,16905.0,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th..."


In [48]:
data["vis"] = 0;
visted = [0] * len(data)

In [49]:
for i in range(len(data)):
  data.loc[i, 'vis'] = i

In [50]:
ans = 0
y, index = 0, 0
for i in range(len(data)):
    user_id_len = len(str(data['user_id'][i]).split(","))
    review_title_len = len(str(data['review_title'][i]).split(","))
    if review_title_len != user_id_len:
        if (abs(review_title_len - user_id_len) > y):
            y = abs(review_title_len - user_id_len)
            index = i
        ans += 1
print(ans)
x = (str(data['review_title'][index]).split(","))
print(len(x))
print(index)
okians= 1
print(data["product_id"][index])
for i in x:
    print(i)
for i in x:
    oki = i
    if len(oki) >= 1 and oki[0] >= "A" and oki[0] <= "Z":
        print(i, " - ", okians)
        okians += 1
y = (str(data['user_id'][index]).split(","))
print("=======================")
print(len(y))
for i in y:
    print(i)

476
14
767
B08YD264ZS
Good
Quite Good
 just a little too costly
 but worth it
Best in this budget segment
Table is good

 wood is hard
 legs are strong
 but plastic frame is broken at one corner
The product quality is good. The size is small. It is stable not bending while working on table.
Good 👍
Perfect for WFH !
Best buy
Good  -  1
Quite Good  -  2
Best in this budget segment  -  3
Table is good  -  4
The product quality is good. The size is small. It is stable not bending while working on table.  -  5
Good 👍  -  6
Perfect for WFH !  -  7
Best buy  -  8
8
AHXVJ4RECEDVRCX2R7BYOMRO7KJQ
AEUNZGZ7IQFCJEFHU647HB57FC2Q
AEUWYI55HVW2GO4GRLWK4PWCTPLQ
AEDRDM7OTIWIAOWELAEAITODC4EA
AEDZ4OLR66LZO57XWMR6F43K736A
AEIXRXVWCR62IELG44BI5F7ZZUSQ
AFMCGE5U34NNKT2AGRY5TPX4OHKQ
AEFVX5GYQ6Y5MQSA25IP2FM2ZKTA


In [51]:
ans = 0
y, index = 0, 0
i = 0
while(i < len(data)):
    lis_user = str(data['user_id'][i]).split(",")
    lis_review = str(data['review_title'][i]).split(",")
    if len(lis_user) <= len(lis_review) and visted[data["vis"][i]] == 0:
        if len(lis_user) > 1:
            copy = data.iloc[[i]]
            new = pd.concat([copy] * (len(lis_user) - 1), ignore_index=True)
            data = pd.concat([data.iloc[:i+1], new, data.iloc[i+1:]], ignore_index=True)

            d = i
            for j in range(len(lis_user)):
                data.loc[d, 'user_id'] = lis_user[j]
                d += 1
            tmp = ""
            okicheck = 0
            j, d = 0, i
            m = []
            for j in lis_review:
                if len(j) >= 1:
                  m.append(j)
            lis_review = m
            j, d = 0, i
            while (j < len(lis_review)):
                x = lis_review[j]
                if x[0] >= "A" and x[0] <= "Z":
                    tmp = x
                    for z in range(j + 1, len(lis_review)):
                        y = lis_review[z]
                        if (y[0] >= "A" and y[0] <= "Z") == False:
                            tmp = tmp + " " + y
                        else:
                            j = z - 1
                            break
                    data.loc[d, 'review_title'] = tmp
                    d += 1
                j += 1
            index = d

        visted[data["vis"][i]] = 1
    i += 1


In [52]:

ans = 0
for i in range(len(data)):
  if data["product_id"][i] == "B08YD264ZS":
    print(visted[data["vis"][i]], data["user_id"][i], data["review_title"][i])
    ans += 1


1 AHXVJ4RECEDVRCX2R7BYOMRO7KJQ Good
1 AEUNZGZ7IQFCJEFHU647HB57FC2Q Quite Good  just a little too costly  but worth it
1 AEUWYI55HVW2GO4GRLWK4PWCTPLQ Best in this budget segment
1 AEDRDM7OTIWIAOWELAEAITODC4EA Table is good  wood is hard  legs are strong  but plastic frame is broken at one corner
1 AEDZ4OLR66LZO57XWMR6F43K736A The product quality is good. The size is small. It is stable not bending while working on table.
1 AEIXRXVWCR62IELG44BI5F7ZZUSQ Good 👍
1 AFMCGE5U34NNKT2AGRY5TPX4OHKQ Perfect for WFH !
1 AEFVX5GYQ6Y5MQSA25IP2FM2ZKTA Best buy


In [53]:
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11454 entries, 0 to 11453
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        11454 non-null  object
 1   category          11454 non-null  object
 2   discounted_price  11454 non-null  object
 3   actual_price      11454 non-null  object
 4   rating            11454 non-null  object
 5   rating_count      11454 non-null  object
 6   user_id           11454 non-null  object
 7   review_title      11454 non-null  object
 8   review_content    11454 non-null  object
 9   vis               11454 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 895.0+ KB


,product_id,category,discounted_price,actual_price,rating,rating_count,user_id,review_title,review_content,vis
0,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,AG3D6O4STAQKAY2UVGEUV46KN35Q,Satisfied,Looks durable Charging is fine tooNo complains...,0
1,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,AHMY5CWJMMK5BJRBBSNLYT3ONILA,Charging is really fast,Looks durable Charging is fine tooNo complains...,0
2,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,AHCTC6ULH4XB6YHDY6PCH2R772LQ,Value for money,Looks durable Charging is fine tooNo complains...,0
3,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,Product review,Looks durable Charging is fine tooNo complains...,0
4,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,Good quality,Looks durable Charging is fine tooNo complains...,0
...,...,...,...,...,...,...,...,...,...,...
11449,B01486F4G6,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,2863.0,3690.0,4.3,6987.0,AHXCDNSXAESERITAFELQABFVNLCA,Excellent product✌,It does it job perfectly..only issue is temp c...,1464
11450,B01486F4G6,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,2863.0,3690.0,4.3,6987.0,AGRZD6CHLCUNOLMMIMIHUCG7PIFA,A good product for household use मुझे बिल्कुल ...,It does it job perfectly..only issue is temp c...,1464
11451,B01486F4G6,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,2863.0,3690.0,4.3,6987.0,AFQZVGSOSOJHKFQQMCEI4725QEKQ,Best product,It does it job perfectly..only issue is temp c...,1464
11452,B01486F4G6,Home&Kitchen|Kitchen&HomeAppliances|SmallKitch...,2863.0,3690.0,4.3,6987.0,AEALVGXXIP46OZVXKRUXSDWZJMEA,Good,It does it job perfectly..only issue is temp c...,1464


In [54]:
li, li2 = [], []
for i in range(len(data)):
    li.append(data["user_id"][i])
    li2.append(data["review_title"][i])

seli = set(li)
seli2 = set(li2)


li, li2 = list(seli), list(seli2)
d, d2 = 1, 1
dicli = {}
for i in li:
    dicli[i] = d
    d += 1

for i in range(len(data)):
    data.loc[i, 'user_id'] = dicli[data["user_id"][i]]


In [55]:
# import sys
# sys.stdout = open('output.txt', 'w')

# for i in li2:
#   print(i)

In [56]:
data.head()


,product_id,category,discounted_price,actual_price,rating,rating_count,user_id,review_title,review_content,vis
0,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,3251,Satisfied,Looks durable Charging is fine tooNo complains...,0
1,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,220,Charging is really fast,Looks durable Charging is fine tooNo complains...,0
2,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,6266,Value for money,Looks durable Charging is fine tooNo complains...,0
3,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,4951,Product review,Looks durable Charging is fine tooNo complains...,0
4,B07JW9H4J1,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,4.2,24269.0,7542,Good quality,Looks durable Charging is fine tooNo complains...,0


In [57]:
print(len(li), len(li2), len(data))

9020 6450 11454


In [58]:
for i in range(len(data)):
  x = data["category"][i]
  x = x.split("|")
  data.loc[i, 'category'] = x[-1]

In [59]:
data.head()

,product_id,category,discounted_price,actual_price,rating,rating_count,user_id,review_title,review_content,vis
0,B07JW9H4J1,USBCables,399.0,1099.0,4.2,24269.0,3251,Satisfied,Looks durable Charging is fine tooNo complains...,0
1,B07JW9H4J1,USBCables,399.0,1099.0,4.2,24269.0,220,Charging is really fast,Looks durable Charging is fine tooNo complains...,0
2,B07JW9H4J1,USBCables,399.0,1099.0,4.2,24269.0,6266,Value for money,Looks durable Charging is fine tooNo complains...,0
3,B07JW9H4J1,USBCables,399.0,1099.0,4.2,24269.0,4951,Product review,Looks durable Charging is fine tooNo complains...,0
4,B07JW9H4J1,USBCables,399.0,1099.0,4.2,24269.0,7542,Good quality,Looks durable Charging is fine tooNo complains...,0


In [60]:
li, li2 = [], []
for i in range(len(data)):
    li.append(data["category"][i])

seli = set(li)


li, li2 = list(seli), list(seli2)
d, d2 = 1, 1
dicli = {}
for i in li:
    dicli[i] = d
    d += 1

for i in range(len(data)):
    data.loc[i, 'category'] = dicli[data["category"][i]]


In [61]:
data.head()

,product_id,category,discounted_price,actual_price,rating,rating_count,user_id,review_title,review_content,vis
0,B07JW9H4J1,18,399.0,1099.0,4.2,24269.0,3251,Satisfied,Looks durable Charging is fine tooNo complains...,0
1,B07JW9H4J1,18,399.0,1099.0,4.2,24269.0,220,Charging is really fast,Looks durable Charging is fine tooNo complains...,0
2,B07JW9H4J1,18,399.0,1099.0,4.2,24269.0,6266,Value for money,Looks durable Charging is fine tooNo complains...,0
3,B07JW9H4J1,18,399.0,1099.0,4.2,24269.0,4951,Product review,Looks durable Charging is fine tooNo complains...,0
4,B07JW9H4J1,18,399.0,1099.0,4.2,24269.0,7542,Good quality,Looks durable Charging is fine tooNo complains...,0


In [62]:
print(len(data['category'].unique()))

207


In [63]:
data = data.drop_duplicates()

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11454 entries, 0 to 11453
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        11454 non-null  object
 1   category          11454 non-null  object
 2   discounted_price  11454 non-null  object
 3   actual_price      11454 non-null  object
 4   rating            11454 non-null  object
 5   rating_count      11454 non-null  object
 6   user_id           11454 non-null  object
 7   review_title      11454 non-null  object
 8   review_content    11454 non-null  object
 9   vis               11454 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 895.0+ KB


In [65]:
data.drop(columns = ['vis'], inplace = True)

In [66]:
data.head()

,product_id,category,discounted_price,actual_price,rating,rating_count,user_id,review_title,review_content
0,B07JW9H4J1,18,399.0,1099.0,4.2,24269.0,3251,Satisfied,Looks durable Charging is fine tooNo complains...
1,B07JW9H4J1,18,399.0,1099.0,4.2,24269.0,220,Charging is really fast,Looks durable Charging is fine tooNo complains...
2,B07JW9H4J1,18,399.0,1099.0,4.2,24269.0,6266,Value for money,Looks durable Charging is fine tooNo complains...
3,B07JW9H4J1,18,399.0,1099.0,4.2,24269.0,4951,Product review,Looks durable Charging is fine tooNo complains...
4,B07JW9H4J1,18,399.0,1099.0,4.2,24269.0,7542,Good quality,Looks durable Charging is fine tooNo complains...


## Encode review rating
-  Target variable: rating (divide into 3 bin: unsatisfied  <=2, normal < 4, satisfied >=4)

In [67]:
print(data['rating'].isnull().sum())
print(type(data['rating'].iloc[0]))

0
<class 'float'>


In [68]:
def encode_rating(rating: float) -> int:
    try:
        rating = float(rating)
    except:
        return -1
    if rating <= 0:
        return -1
    elif rating <= 2:
        return 0
    elif rating < 4:
        return 1
    elif rating > 5:
        return -1
    else:
        return 2

In [69]:
data['encode_review_raing'] = np.vectorize(encode_rating)(data['rating'])

In [70]:
# drop row with -1 in col rating
data = data[data['encode_review_raing'] != -1]

## Text processing
- Normalise text (rm stop words, hyperlink)
- remove emoji, emoticons
- remove non ascii characters

In [71]:
# def text_normalize(text):
#     #lowercasing
#     text = text.lower()
    
#     # hyperlinks removal
#     text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    
#     # remove punctuation
#     text = re.sub(r'[^\w\s]', ' ', text)
    
#     #Remove stopwords
#     stopwords_txt = pd.read_csv('../data/stop_words_english.csv', header=None)
#     stop_words = stopwords_txt[0].tolist()
#     words = text.split()
#     words = [word for word in words if word not in stop_words]
#     text = ' '.join(words)                                          # Space handling
#     return text

# # remove emoticons to using emot
# def handle_emoticons(text, remove_emoticon=True):
#     dict_emoticons = dict(zip(emot_obj.emoticons(text)['value'], emot_obj.emoticons(text)['mean']))
#     res_emoticons =  dict(sorted(dict_emoticons.items(), key = lambda kv:len(kv[1]), reverse=True))
#     for emoticon, mean in res_emoticons.items():
#         if remove_emoticon:
#             text = text.replace(emoticon, "")
#         else:
#             text = text.replace(emoticon, mean)
#     return text

# def handle_emojis(text, remove_emoji=True):
#     for emoji, mean in zip(emot_obj.emoji(text)['value'], emot_obj.emoji(text)['mean']):
#         if remove_emoji:
#             text = text.replace(emoji, "")
#         else:
#             text = text.replace(emoji, mean)
#     return text

# def keep_ascii_alnum(text):
#     # 1) Strip out any char that's not a letter, digit, or whitespace
#     cleaned = re.sub(r'[^A-Za-z0-9\s]+', '', text)
    
#     # 2) Collapse runs of whitespace into a single space, and trim ends
#     cleaned = re.sub(r'\s+', ' ', cleaned).strip()
    
#     return str(cleaned)

In [72]:
data['review_title'] = np.vectorize(text_normalize)(data['review_title'])
data['review_title'] = np.vectorize(handle_emoticons)(data['review_title'])
data['review_title'] = np.vectorize(handle_emojis)(data['review_title'])
data['review_title'] = np.vectorize(keep_ascii_alnum)(data['review_title'])

data['review_content'] = np.vectorize(text_normalize)(data['review_content'])
data['review_content'] = np.vectorize(handle_emoticons)(data['review_content'])
data['review_content'] = np.vectorize(handle_emojis)(data['review_content'])
data['review_content'] = np.vectorize(keep_ascii_alnum)(data['review_content'])

## Lematization

In [73]:
# source: https://www.geeksforgeeks.org/python-lemmatization-with-nltk/

# Lemmatization
nlp = spacy.load("en_core_web_sm")  

# def lemmatize_text(text):
#     doc = nlp(text)
#     return " ".join([token.lemma_ for token in doc])

sample_text = "The quick brown foxes are jumping over the lazy dogs."
print("lemmatized text:", lemmatize_text(sample_text))

NameError: name 'lemmatize_text' is not defined

In [ ]:
data['review_title'] = np.vectorize(lemmatize_text)(data['review_title'])
data['review_content'] = np.vectorize(lemmatize_text)(data['review_content'])


In [ ]:
data.isnull().count()

product_id             11446
category               11446
discounted_price       11446
actual_price           11446
rating                 11446
rating_count           11446
user_id                11446
review_title           11446
review_content         11446
encode_review_raing    11446
dtype: int64

## Tokenize the col 'title and review' (omit for now)

In [ ]:
# tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
#                                reduce_len=True)

# # df['title and review'] = np.vectorize(tokenizer.tokenize)(df['title and review'])
# df['title and review'] = df['title and review'].apply(tokenizer.tokenize)

In [ ]:
data.isna().sum()

product_id             0
category               0
discounted_price       0
actual_price           0
rating                 0
rating_count           0
user_id                0
review_title           0
review_content         0
encode_review_raing    0
dtype: int64

## Final remark
- output: amazon-cleaned.csv
- output: dic-userid-username.json ???
- output: dic-reviewid-userid (primary key will be review_id)

In [ ]:
output_file_path = '../data/amazon-cleaned.csv'  # Đường dẫn và tên file
data.to_csv(output_file_path, index=False)

output_file_path

'amazon-cleaned.csv'

In [ ]:
type(data)

pandas.core.frame.DataFrame